In [1]:
# RPA Week #2 challenge – https://developer.automationanywhere.com/blog/challenge-page-intro-supply-chain

import rpa as r
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import string
import requests
import pandas as pd
import time
import sys

r.init()

# configurable variables - WebDriver (ChromeDriver 92.0.4515.107)
browser = webdriver.Chrome(('./chromedriver'))

# configurable variables - URL
url_PurchaseOrderTracking = 'https://developer.automationanywhere.com/challenges/AutomationAnywhereLabs-POTrackingLogin.html'
url_SupplyChainManagement = 'https://developer.automationanywhere.com/challenges/automationanywherelabs-supplychainmanagement.html'

# configurable variables - Credentials
email = 'admin@procurementanywhere.com'
password = 'paypacksh!p'

# configurable variables - CSS Selector/ XPaths in 'Purchase Order Tracking' webpage
CSS_Selector_Button_POT_SignIn = 'button.btn.btn-lg.btn-primary.btn-block.text-uppercase'
XPath_POT_Error = '//*[@id="myModal"]'
XPath_POT_Cookies_Accept = '//*[@id="onetrust-accept-btn-handler"]'
XPath_POT_inputEmail = '//*[@id="inputEmail"]'
XPath_POT_inputPassword = '//*[@id="inputPassword"]'
XPath_POT_SearchBar = '//*[@id="dtBasicExample_filter"]/label/input'
XPath_POT_State = '//*[@id="dtBasicExample"]/tbody/tr/td[5]'
XPath_POT_ShipDate = '//*[@id="dtBasicExample"]/tbody/tr/td[7]'
XPath_POT_OrderTotal = '//*[@id="dtBasicExample"]/tbody/tr/td[8]'
XPath_Button_POT_Logout = '/html/body/header/div/div/span[2]/a'

# configurable variables - HTML attributes in 'Supply Chain Management' webpage
HTML_Class_Button_TerritoryExcel = 'btn'
HTML_ID_PONumber_StartWith = 'PONumber'
HTML_ID_shipDate_StartWith = 'shipDate'
HTML_ID_orderTotal_StartWith = 'orderTotal'
HTML_ID_agent_StartWith = 'agent'
HTML_ID_submitbutton = 'submitbutton'
HTML_ID_ResultTime = 'processing-time'
HTML_ID_ResultAccuracy = 'accuracy'

# configurable variables - filename of result
fileName = 'results_screenshot.png'

# access 'Purchase Order Tracking' webpage
browser.get(url_PurchaseOrderTracking)
wait = WebDriverWait(browser, 5)
wait.until(EC.presence_of_element_located((By.XPATH, XPath_POT_Cookies_Accept))) #wait for cookies accept prompt to appear
browser.find_element_by_xpath(XPath_POT_Cookies_Accept).click()
browser.find_element_by_xpath(XPath_POT_inputEmail).send_keys(email)
browser.find_element_by_xpath(XPath_POT_inputPassword).send_keys(password)
browser.find_element_by_css_selector(CSS_Selector_Button_POT_SignIn).click()

# error handling - failed to login
error = False
try:
    wait.until(EC.text_to_be_present_in_element((By.XPATH, XPath_POT_Error),'error_outline'))
    if 'error_outline' in browser.find_element_by_xpath(XPath_POT_Error).text:
        error = True
except:
    pass

if error:
    print('Failed to login. Username or password might be incorrect. Please try again.')
    # close 'Purchase Order Tracking' webpage
    browser.close()
    r.close()
    sys.exit()
      
# access 'Supply Chain Management' webpage 
r.url(url_SupplyChainManagement)
start_time = time.time()

# download Agent Territory Spreadsheet from 'Supply Chain Management' webpage
TerritoryDF = pd.read_excel(r.read('//*[@class="{}"]/@href'.format(HTML_Class_Button_TerritoryExcel)))
TerritoryDF = dict(zip(TerritoryDF[TerritoryDF.columns[0]], TerritoryDF[TerritoryDF.columns[1]]))

# count total number of PO in 'Supply Chain Management' webpage
totalPO = r.count('//*[contains(@id,"{}")]'.format(HTML_ID_PONumber_StartWith))

for num in range(1,totalPO+1):
    # get a PO Number from 'Supply Chain Management' webpage
    PONumber = r.read('//*[@id="{}"]'.format(HTML_ID_PONumber_StartWith + str(num)))
    
    # get State, Ship Date and Order Total from 'Purchase Order Tracking' webpage
    browser.find_element_by_xpath(XPath_POT_SearchBar).send_keys(PONumber)
    State = browser.find_element_by_xpath(XPath_POT_State).text
    ShipDate = browser.find_element_by_xpath(XPath_POT_ShipDate).text    
    OrderTotal = browser.find_element_by_xpath(XPath_POT_OrderTotal).text[1:]    
    browser.find_element_by_xpath(XPath_POT_SearchBar).clear()
    
    # type in Ship Date into 'Supply Chain Management' webpage
    r.type('//*[@id="{}"]'.format(HTML_ID_shipDate_StartWith + str(num)), ShipDate)
    
    # type in Order Total into 'Supply Chain Management' webpage
    r.type('//*[@id="{}"]'.format(HTML_ID_orderTotal_StartWith + str(num)), OrderTotal)
    
    # select in Assigned Agent into 'Supply Chain Management' webpage
    domAgent = 'document.getElementById("{}").value = "{}"'.format(HTML_ID_agent_StartWith + str(num),TerritoryDF[State])
    r.dom(domAgent)

# click submit button in 'Supply Chain Management' webpage
r.click('//*[@id="{}"]'.format(HTML_ID_submitbutton))
print("--- %s seconds ---" % (time.time() - start_time))

# print results
Time = r.read('//*[@id="{}"]'.format(HTML_ID_ResultTime))
Accuracy = r.read('//*[@id="{}"]'.format(HTML_ID_ResultAccuracy))
if r.read('//*[@id="{}"]'.format(HTML_ID_ResultAccuracy)) != '100.00%':
    print ('Timing= {} / Accuracy= {}. Input error. Please try again'.format(Time, Accuracy))
else:
    print ('Timing= {} / Accuracy= {}'.format(Time, Accuracy))
    r.snap('page', fileName) #take screenshot of result
    
# click logout button and close 'Purchase Order Tracking' webpage
browser.find_element_by_xpath(XPath_Button_POT_Logout).click()
browser.close()

r.close()

--- 3.812594175338745 seconds ---
Timing= 3.216 seconds / Accuracy= 100.00%


True